In [1]:
import os
import mne
import pandas as pd
import numpy as np
import janitor
import warnings
from tqdm.notebook import tqdm

warnings.filterwarnings("ignore", category=RuntimeWarning)

data_path = '/home/owner/Documents/DEV/BrainLabyrinth/data/Ymaze_exp'
final_dataset_path = '/home/owner/Documents/DEV/BrainLabyrinth/data/final_dataset'

In [2]:
# # from sklearn.decomposition import PCA as sklearnPCA
# import scipy.signal as signal
# from mne.preprocessing import ICA


# for user_id in tqdm(os.listdir(data_path)):
#     person_dir = os.path.join(data_path, user_id)
#     for file in os.listdir(person_dir):
#         file_path = os.path.join(person_dir, file)

#         if file.endswith(".vhdr"):
#             # Read the EEG data
#             raw = mne.io.read_raw_brainvision(file_path, preload=True, ignore_marker_types=True, verbose=False)
            
#             # Apply preprocessing steps

#             # 1. Resample to 500 Hz
#             raw.resample(500, npad="auto")  # Resample to 500 Hz
            
#             # 2. Filter high-pass at 0.1 Hz (for low-frequency noise removal)
#             raw.filter(l_freq=0.1, h_freq=30)  # High-pass filter (low-frequency cutoff at 0.1 Hz)
            
#             # 3. Notch filter at 50 Hz (to remove power line noise)
#             raw.notch_filter(freqs=50, picks='all')  # Notch filter for 50 Hz

#             # 4. ICA for blink removal
#             ica = ICA(n_components=20, method='fastica', max_iter=2000, random_state=97)
#             ica.fit(raw, picks='eeg')  # Fit to EEG channels excluding EOG
            
#             # Detect EOG artifacts
#             eog_indices, eog_scores = ica.find_bads_eog(raw, ch_name='EOG', threshold=2.0)
#             ica.exclude = eog_indices
            
#             # Apply ICA cleaning
#             raw = ica.apply(raw)
#             raw.drop_channels(['EOG'])  # Remove auxiliary EOG channel after cleaning

#             # 4. Detrend the data using scipy.signal.detrend
#             raw._data = signal.detrend(raw._data, axis=1)  # Detrend along the time axis (axis=1)
            
#             # Continue with the rest of your pipeline as you have it
#             # Find the corresponding .vmrk file
#             vmrk_file = file.replace('.vhdr', '.vmrk')
#             vmrk_file_path = os.path.join(person_dir, vmrk_file)

#             if os.path.exists(vmrk_file_path):
#                 # Read the annotations (markers) from the .vmrk file
#                 annotations = mne.annotations.read_annotations(vmrk_file_path)
#             # Convert raw data to DataFrame
#             time_series = raw.to_data_frame()

#             # Extract markers (annotations)
#             marker_times = annotations.onset  # In seconds
#             marker_labels = annotations.description  # The marker labels

#             # Create a DataFrame for the markers
#             markers_df = pd.DataFrame({
#                 'event_id': np.arange(len(marker_labels)), 
#                 'start': marker_times - 3.0,
#                 'end': marker_times + 1.0,
#                 'marker': marker_labels
#             })
            
#             markers_df = markers_df[
#                 ~markers_df.marker.isin([
#                     'Marker/Impedance', 'New Segment/'#, 'Stimulus/2'
#                 ])
#             ]
#             # markers_df.marker = markers_df.marker.replace({'Stimulus/2': 'Stimulus/P'})

#             # Display the first few rows of both DataFrames
#             # Merge markers with EEG data
#             time_series['time'] = time_series['time'].round(3)  # Round times to 3 decimal places for matching

#             merged_df = janitor.conditional_join(
#                 markers_df, 
#                 time_series,
#                 ('start', 'time', '<='),
#                 ('end', 'time', '>='),
#                 how='left',
#                 df_columns=['event_id', 'marker']
#             )
#             # display(merged_df.groupby('event_id').count())
#             merged_path = os.path.join(final_dataset_path, f"{user_id}.parquet")
#             merged_df.to_parquet(merged_path)

In [3]:
import os
ok = os.listdir('/home/owner/Documents/DEV/BrainLabyrinth/data/final_dataset copy')

cleaned_dir = '/home/owner/Documents/DEV/BrainLabyrinth/data/final_dataset'
for file in os.listdir(cleaned_dir):
    if file not in ok:
        os.remove(os.path.join(cleaned_dir, file))

In [4]:
parquet_directory = "/home/owner/Documents/DEV/BrainLabyrinth/data/final_dataset"

parquet_files = [
    os.path.join(parquet_directory, f)
    for f in os.listdir(parquet_directory)
    if f.endswith(".parquet")
]

def next_direction(marker: str, last_dir: str) -> str:
    """
    Return the new direction ("Left" or "Right" or None) based on:
      - Stimulus/1 => "Left"
      - Stimulus/2 => "Right"
      - Stimulus/A => flip last_dir (None -> None)
      - Stimulus/P => same as last_dir (None -> None)
      - else => keep last_dir
    """
    base_map = {
        "Stimulus/1": "Left",
        "Stimulus/2": "Right",
    }

    if marker in base_map:
        return base_map[marker]
    if marker == "Stimulus/A":
        if last_dir is None:
            return None
        return "Right" if last_dir == "Left" else "Left"
    if marker == "Stimulus/P":
        return last_dir  # could be None

    return last_dir

def label_whole_block(pdf, event_col="event_id", marker_col="orig_marker", time_col="time"):
    """
    1) Remove 'Stimulus/s16' rows from 'orig_marker'.
    2) Sort by 'time'.
    3) Sort blocks by earliest time in each event_id.
    4) For each block, determine new direction from 'orig_marker'.
       Save the 'prev_marker'.
    5) Drop first-turn events (prev_marker == None).
    6) Rename 'direction' -> 'marker'.
    """
    # 1) Remove 'Stimulus/s16'
    pdf = pdf.loc[pdf[marker_col] != "Stimulus/s16"].copy()

    # 2) Sort by time
    pdf = pdf.sort_values(time_col)

    # 3) Sort blocks by earliest time
    earliest_times = pdf.groupby(event_col)[time_col].min().sort_values()
    sorted_event_ids = earliest_times.index.tolist()

    # Single marker per block
    block_marker_map = (
        pdf.groupby(event_col)[marker_col]
           .first()
           .to_dict()
    )

    block_dir_map = {}
    block_prev_map = {}

    last_direction = None

    for e_id in sorted_event_ids:
        event_marker = block_marker_map[e_id]
        block_prev_map[e_id] = last_direction

        direction = next_direction(event_marker, last_direction)
        block_dir_map[e_id] = direction

        last_direction = direction

    pdf["direction"] = pdf[event_col].map(block_dir_map)
    pdf["prev_marker"] = pdf[event_col].map(block_prev_map)

    # Drop first-turn events
    pdf = pdf.loc[pdf["prev_marker"].notna()].copy()

    # Rename 'direction' -> 'marker'
    pdf.rename(columns={"direction": "marker"}, inplace=True)

    return pdf


all_dfs = []
for file_path in tqdm(parquet_files):
    # Read the Parquet file
    df = pd.read_parquet(file_path)

    # RENAME original 'marker' => 'orig_marker' so we don't conflict later
    df.rename(columns={"marker": "orig_marker"}, inplace=True)

    # Label each block
    labeled_df = label_whole_block(df)
  
    df.drop(columns=["orig_marker"], inplace=True)

    # Make event_id unique per file
    file_name = os.path.splitext(os.path.basename(file_path))[0]
    labeled_df["event_id"] = labeled_df["event_id"].astype(str) + "_" + file_name

    all_dfs.append(labeled_df)

combined_df = pd.concat(all_dfs, ignore_index=True)

# Encode event_id
unique_events = combined_df["event_id"].unique()
event_map = {ev: i for i, ev in enumerate(unique_events)}
combined_df["event_id"] = combined_df["event_id"].map(event_map).astype("int64")

# Write to Parquet with no duplicates
output_file = "/home/owner/Documents/DEV/BrainLabyrinth/data/combined.parquet"
combined_df.to_parquet(output_file)
print("Saved to:", output_file)


  0%|          | 0/21 [00:00<?, ?it/s]

Saved to: /home/owner/Documents/DEV/BrainLabyrinth/data/combined.parquet


In [5]:
combined_df


,event_id,orig_marker,time,Fp1,Fpz,Fp2,F7,F3,Fz,F4,...,PO6,FT7,FT8,TP7,TP8,PO7,PO8,Oz,marker,prev_marker
0,0,Stimulus/A,16.458,17.170373,13.906436,21.251943,0.049140,6.744807,-11.924581,-0.963751,...,14.079291,11.501145,1.636257,5.387385,6.445890,14.401535,13.353615,8.559186,Right,Left
1,0,Stimulus/A,16.460,18.195809,14.819681,21.679231,1.756323,7.864059,-11.064782,-0.080928,...,13.092330,12.683138,3.162260,6.085818,8.821207,12.746989,12.518362,5.916264,Right,Left
2,0,Stimulus/A,16.462,19.208807,15.713978,22.097658,3.589434,8.999892,-10.254515,0.710523,...,12.511693,13.800554,4.745254,6.843331,11.128809,10.785815,12.086690,3.017775,Right,Left
3,0,Stimulus/A,16.464,20.146230,16.545905,22.489029,5.424029,10.097336,-9.551693,1.345182,...,12.429921,14.822983,6.309243,7.631156,13.276480,8.708844,12.141982,0.159280,Right,Left
4,0,Stimulus/A,16.466,20.946355,17.271059,22.827998,7.140907,11.096112,-9.006329,1.771387,...,12.871215,15.728430,7.780457,8.422234,15.192707,6.723113,12.701932,-2.368745,Right,Left
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5544003,2771,Stimulus/A,956.836,-18.873559,-24.227101,-13.486943,-0.403397,-11.983540,-16.557347,-17.518873,...,10.332691,0.287628,-6.543679,5.453187,0.753634,3.273086,10.787134,5.025360,Left,Right
5544004,2771,Stimulus/A,956.838,-18.704103,-24.969565,-14.585048,-1.047141,-11.943701,-16.457730,-17.959561,...,10.351784,-0.944356,-7.749362,3.527842,-0.227262,1.671753,10.779771,4.207392,Left,Right
5544005,2771,Stimulus/A,956.840,-18.061667,-25.302962,-15.126122,-1.543149,-11.638041,-16.168372,-18.234343,...,10.233550,-2.014752,-9.047787,1.966788,-1.368454,0.333598,10.611117,3.436481,Left,Right
5544006,2771,Stimulus/A,956.842,-17.023192,-25.220666,-15.079919,-1.852834,-11.103140,-15.743369,-18.338312,...,10.048173,-2.819302,-10.318089,0.896298,-2.548241,-0.638066,10.358187,2.779875,Left,Right


In [6]:
# Define the directory containing the Parquet files
parquet_directory = '/home/owner/Documents/DEV/BrainLabyrinth/data/combined.parquet'

# Read the Parquet files into a Dask DataFrame
df = pd.read_parquet(parquet_directory)

# Compute the shape of the Dask DataFrame
rows, cols = df.shape
rows_computed = rows

# Print the shape
print(f"Shape of the final dataset: ({rows_computed}, {cols})")

Shape of the final dataset: (5544008, 68)
